In [515]:
import numpy as np

In [516]:
import numpy as np
import pandas as pd
import pymc3 as pm
import arviz as az
import theano.tensor as tt
import scipy as sp
from scipy import stats

In [517]:
data = []
with open('data.txt' , 'r') as f:
    for line in f.readlines():
        data.append(line.strip('\n'))

In [518]:
print(data)

['## Data', 'lsat = Dict{Symbol, Any}(', '  :cumulative Frequency =>', '    [3, 9, 11, 22, 23, 24, 27, 31, 32, 40, 40, 56, 56, 59, 61, 76, 86, 115, 129,', '     210, 213, 241, 256, 336, 352, 408, 429, 602, 613, 674, 702, 1000],', '  :response =>', '    [0 0 0 0 0', '     0 0 0 0 1', '     0 0 0 1 0', '     0 0 0 1 1', '     0 0 1 0 0', '     0 0 1 0 1', '     0 0 1 1 0', '     0 0 1 1 1', '     0 1 0 0 0', '     0 1 0 0 1', '     0 1 0 1 0', '     0 1 0 1 1', '     0 1 1 0 0', '     0 1 1 0 1', '     0 1 1 1 0', '     0 1 1 1 1', '     1 0 0 0 0', '     1 0 0 0 1', '     1 0 0 1 0', '     1 0 0 1 1', '     1 0 1 0 0', '     1 0 1 0 1', '     1 0 1 1 0', '     1 0 1 1 1', '     1 1 0 0 0', '     1 1 0 0 1', '     1 1 0 1 0', '     1 1 0 1 1', '     1 1 1 0 0', '     1 1 1 0 1', '     1 1 1 1 0', '     1 1 1 1 1],', '  :N => 1000', ')', '']


In [519]:
choices = []
print()
for student in data[6 : len(data) - 3]:
    choice = []
    student_rev = student[::-1]
    student_split = student.split(' ')
    for c in student_split[-5 : ]:
        if '[' in c:
            choice.append(int(c[1]))
        elif ']' in c:
            choice.append(int(c[0]))
        else:
            choice.append(int(c))
    choices.append(choice)
        

In [520]:
cumulative = []

cum1 = data[3].strip(' ')
cum2 = data[4].strip(' ')[:-1]
cum1 = cum1.strip('[')
cum2 = cum2.strip(']')
cum = cum1 + cum2
cum_split = cum.split(',')

for c in cum_split:
    cumulative.append(int(c))

cumulative = np.array(cumulative)

Rasch Model

$\Pr(X_{ni}=1) = \frac{e^{\beta_n - \delta_i}}{1 + e^{\beta_n - \delta_i}}$

$\beta_n$ = ability of student n

$\delta_i$ = difficulty of item i

https://towardsdatascience.com/a-bayesian-approach-to-rasch-models-item-response-theory-cc08805cbb37

In [521]:
data_matrix = []

for idx in range(len(cumulative)):
    if idx == 0:
        for i in range(cumulative[idx]):
            data_matrix.append(choices[idx])
    else: 
        for i in range(cumulative[idx] - cumulative[idx - 1]):
            data_matrix.append(choices[idx])

data_matrix  = np.array(data_matrix)

In [544]:
alpha_initial = np.random.normal(0 , 100 , size = (5,))
alpha_initial -= alpha_initial.mean()
beta_initial = np.random.uniform(0 , 100)
theta_initial = np.random.normal( 0 , 1 , size = (1000 , ))


In [545]:
alpha_initial

array([  43.35744674,   81.74231917, -105.85506819,  -11.74940181,
         -7.49529592])

In [523]:
proposal_alpha = lambda x: np.random.multivariate_normal(x , np.eye(x.shape[0]))
proposal_beta = lambda x: np.random.normal(x , 0.1)
proposal_theta = lambda x: np.random.multivariate_normal(x , np.eye(x.shape[0]))

In [536]:
def log_likelihood(data_matrix , alpha , beta , theta):

    sum = 0
    for i in range(data_matrix.shape[0]):
        for j in range(data_matrix.shape[1]):

            if data_matrix[i][j] == 0:
                sum += np.log(1 - np.exp(-alpha[j] + beta * theta[i]) / (1 + np.exp(-alpha[j] + beta * theta[i])))
            else:
                sum += np.log(np.exp(-alpha[j] + beta * theta[i]) / (1 + np.exp(-alpha[j] + beta * theta[i])))

            print(sum)
    return sum

In [546]:
log_likelihood(data_matrix , alpha_initial , beta_initial , theta_initial)


0.0
0.0
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-i

/var/folders/x0/r7576wrd5w3dfxhxvxn3v81w0000gn/T/ipykernel_78034/2765117652.py:8: RuntimeWarning: divide by zero encountered in log
  sum += np.log(1 - np.exp(-alpha[j] + beta * theta[i]) / (1 + np.exp(-alpha[j] + beta * theta[i])))


-inf

In [537]:
log_likelihood(data_matrix , alpha_initial , beta_initial , theta_initial)

0.0
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-inf
-

/var/folders/x0/r7576wrd5w3dfxhxvxn3v81w0000gn/T/ipykernel_78034/2765117652.py:8: RuntimeWarning: divide by zero encountered in log
  sum += np.log(1 - np.exp(-alpha[j] + beta * theta[i]) / (1 + np.exp(-alpha[j] + beta * theta[i])))


-inf

In [ ]:
def mcmc_sampler(alpha_initial , beta_initial , theta_initial , num_chains , num_samples, burn_in):

    alpha_samples = []
    beta_samples = []
    theta_samples = []

    for chain_num in range(num_chains):
        alpha , beta , theta = initialize_chain()

        alpha_chain = []
        beta_chain = []
        theta_chain = []

        # we assume proposal distribution to be symmetric
        for step in num_samples:
            
            p_alpha = np.array([sp.stats.norm(0 , 100).pdf(alpha[i]) for i in range(alpha.size)])
            p_alpha = np.product(p_alpha)

            p_beta = sp.stats.norm(0 , 100).pdf(beta)

            p_theta = np.array([sp.stats.norm(0 , 1).pdf(theta[i]) for i in range(theta.size)])
            p_theta = np.product(p_theta)


            proposal_alpha = lambda x: np.random.multivariate_normal(x , np.eye(x.shape[0]))
            proposal_beta = lambda x: np.random.normal(x , 0.1)
            proposal_theta = lambda x: np.random.multivariate_normal(x , np.eye(x.shape[0]))
            
            alpha_new = proposal_alpha(alpha)
            beta_new = proposal_beta(beta)
            theta_new = proposal_theta(theta)

            p_alpha_new = np.array([sp.stats.norm(0 , 100).pdf(alpha_new[i]) for i in range(new_alpha.size)])
            p_alpha_new = np.product(p_alpha_new)

            p_beta_new = sp.stats.norm(0 , 100).pdf(beta_new)

            p_theta_new = np.array([sp.stats.norm(0 , 1).pdf(theta_new[i]) for i in range(new_theta.size)])
            p_theta_new = np.product(p_theta_new)

            likelihood = bernoulli_log_likelihood(data_matrix , alpha , beta , theta)
            likelihood_new = bernoulli_log_likelihood(data_matrix , alpha_new , beta_new , theta_new)

            alpha_ratio = likelihood_new - likelihood + np.log(p_alpha_new) - np.log(p_alpha) \
                np.log(p_beta_new) - np.log(p_beta) + np.log(p_theta_new) - np.log(p_theta)
            alpha_ratio = np.exp(alpha_ratio)

            u = np.random.rand()

            if u >= min(1 , alpha_ratio):
                alpha_chain.append(alpha_new)
                beta_chain.append(beta_new)
                theta_chain.append(theta_new)
                alpha = alpha_new
                beta = beta_new
                theta = theta_new

            else:
                alpha_chain.append(alpha)
                beta_chain.append(beta)
                theta_chain.append(theta)

        alpha_samples.append(alpha_chain)
        beta_samples.append(beta_chain)
        theta_samples.append(theta_chain)


SyntaxError: invalid syntax (4231296731.py, line 46)